In [1]:
from FAFSA.preprocessing import connect_db, build_fafsa

In [3]:
# Connect to example database file
con = connect_db('/mnt/s/FAFSA/l2t_lite.db')

DuckDB version 0.7.1 detected.
Connecting to database...
Connection established! Execution time: 0.5272319316864014 seconds


In [ ]:
# Build example fafsa database from example l2t

build_fafsa(con)